In [188]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

import os

In [189]:
from pathlib import Path

# Configurable parts
SYMBOL = "USDJPY"
SAMPLE_TYPE = "time"
MINUTES = 1
DOLLAR_THRESHOLD = "115009542m"
EVENT_NAME = 'Z-SCORE-W100-1Z'
LABEL_NAME = "TB"

START_DATE = "20240101"
END_DATE = "20241231"

# Build base name
if SAMPLE_TYPE == "dollar":
    RESAMPLED_NAME = f"{SYMBOL}-{DOLLAR_THRESHOLD}-dollar-{START_DATE}-{END_DATE}"
else:
    RESAMPLED_NAME = f"{SYMBOL}-{MINUTES}m-{START_DATE}-{END_DATE}"

# Base directories
BASE_DIR = Path("../data")
RESAMPLED_DIR = BASE_DIR / "resampled"
PROCESSED_DIR = BASE_DIR / "processed"
EVENTS_DIR = BASE_DIR / "events"

# Final paths
RESAMPLED_FILE_PATH = RESAMPLED_DIR / f"{RESAMPLED_NAME}.pkl"
PROCESSED_FILE_PATH = PROCESSED_DIR / f"{RESAMPLED_NAME}-processed.pkl"
EVENT_FILE_PATH = EVENTS_DIR / f"{RESAMPLED_NAME}_{EVENT_NAME}.pkl"

In [190]:
DIRECTION_LABEL_DIR = BASE_DIR / "direction_labels"
DIRECTION_LABEL_DIR.mkdir(parents=True, exist_ok=True)
DIRECTION_LABEL_FILE_PATH = DIRECTION_LABEL_DIR / f"{RESAMPLED_NAME}-{EVENT_NAME}-{LABEL_NAME}.pkl"

In [191]:
%%time
df = pd.read_pickle(PROCESSED_FILE_PATH)
t_events = pd.read_pickle(EVENT_FILE_PATH)

CPU times: user 1.36 ms, sys: 52.8 ms, total: 54.2 ms
Wall time: 59.9 ms


In [192]:
df.columns

Index(['open', 'high', 'low', 'close', 'volume', 'spread', 'close_delta',
       'close_return', 'close_log_return', 'ret_mean_5', 'ret_mean_10',
       'ret_mean_15', 'ret_mean_20', 'log_volume', 'rv5', 'log_rv5',
       'sqrt_rv5', 'rv15', 'log_rv15', 'sqrt_rv15', 'rv50', 'log_rv50',
       'sqrt_rv50', 'ema5', 'ema5_slope', 'close_above_ema5', 'ema20',
       'ema20_slope', 'close_above_ema20', 'ema50', 'ema50_slope',
       'close_above_ema50', 'ema100', 'ema100_slope', 'close_above_ema100',
       'atr14', 'atr60', 'atr120', 'log_atr14', 'atr14_percent',
       'atr14_adjusted_return', 'log_atr60', 'atr60_percent',
       'atr60_adjusted_return', 'log_atr120', 'atr120_percent',
       'atr120_adjusted_return', 'adx14', 'plus_di14', 'minus_di14',
       'bb_upper', 'bb_lower', 'bb_mavg', 'bb_width', 'bb_position',
       'dc20_upper', 'dc20_lower', 'dc20_mid', 'dc20_width',
       'close_above_dc20_mid', 'dc20_breakout', 'dc20_breakdown', 'dc50_upper',
       'dc50_lower', 'dc50_mi

In [193]:
df.loc[t_events].head()

,open,high,low,close,volume,spread,close_delta,close_return,close_log_return,ret_mean_5,...,dom,month,hour_sin,hour_cos,dow_sin,dow_cos,dom_sin,dom_cos,month_sin,month_cos
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-01-02 05:11:00,1.102340,1.102475,1.10234,1.102455,1.185500e+11,0.000023,0.000140,0.000127,0.000127,0.000045,...,2,1,0.965926,0.258819,0.781831,0.62349,0.394356,0.918958,0.5,0.866025
2024-01-02 05:12:00,1.102465,1.102465,1.10239,1.102405,1.712000e+11,0.000022,-0.000050,-0.000045,-0.000045,0.000046,...,2,1,0.965926,0.258819,0.781831,0.62349,0.394356,0.918958,0.5,0.866025
2024-01-02 05:13:00,1.102410,1.102470,1.10240,1.102400,1.297800e+11,0.000022,-0.000005,-0.000005,-0.000005,0.000032,...,2,1,0.965926,0.258819,0.781831,0.62349,0.394356,0.918958,0.5,0.866025
2024-01-02 05:14:00,1.102420,1.102530,1.10238,1.102515,8.331000e+10,0.000019,0.000115,0.000104,0.000104,0.000045,...,2,1,0.965926,0.258819,0.781831,0.62349,0.394356,0.918958,0.5,0.866025
2024-01-02 05:15:00,1.102545,1.102600,1.10249,1.102535,2.250600e+11,0.000022,0.000020,0.000018,0.000018,0.000040,...,2,1,0.965926,0.258819,0.781831,0.62349,0.394356,0.918958,0.5,0.866025


## Calculate Target Volatility (trgt)
This will be used for the calculation of dynamic TP/SL

In [194]:
def get_daily_vol(close,span0=100):
    # daily vol reindexed to close
    df0=close.index.searchsorted(close.index-pd.Timedelta(days=1))
    #bp()
    df0=df0[df0>0]
    #bp()
    df0=(pd.Series(close.index[df0-1],
                   index=close.index[close.shape[0]-df0.shape[0]:]))
    #bp()
    try:
        df0=close.loc[df0.index]/close.loc[df0.values].values-1 # daily rets
    except Exception as e:
        print(e)
        print('adjusting shape of close.loc[df0.index]')
        cut = close.loc[df0.index].shape[0] - close.loc[df0.values].shape[0]
        df0=close.loc[df0.index].iloc[:-cut]/close.loc[df0.values].values-1
    df0=df0.ewm(span=span0).std().rename('dailyVol')
    return df0

In [195]:
from ta.volatility import AverageTrueRange

def get_atr(df, window=60):
    atr = AverageTrueRange(high=df['high'], low=df['low'], close=df['close'], window=window)
    return atr.average_true_range().rename(f"atr{window}")

In [196]:
trgt = get_daily_vol(df['close']).reindex(t_events, method='ffill')

In [197]:
alpha = 1.5  # 你要的寬度倍數
atr = get_atr(df, window=20)                    # 價格單位
atr_rel = (atr / df['close']).shift(1)          # 轉成相對幅度，並用前一根避免洩漏
trgt = (alpha * atr_rel).reindex(t_events, method='ffill')  # 對齊事件

In [198]:
trgt.describe()

count    102686.000000
mean          0.000160
std           0.000106
min           0.000017
25%           0.000087
50%           0.000135
75%           0.000199
max           0.001035
dtype: float64

## Apply Triple Barrier
We use daily volatility to calculate TP/SL, and we set our T1 as 1day

In [199]:
from concurrent.futures import ThreadPoolExecutor
import numpy as np
import pandas as pd

def applyPtSlOnT1(close,events,ptSl,molecule):
    # apply stop loss/profit taking, if it takes place before t1 (end of event)
    events_=events.loc[molecule]
    out=events_[['t1']].copy(deep=True)
    
    pt = ptSl[0] * events_['trgt'] if ptSl[0] > 0 else pd.Series(index=events.index)
    sl = -ptSl[1] * events_['trgt'] if ptSl[1] > 0 else pd.Series(index=events.index)

        
    for loc, t1 in events_['t1'].fillna(close.index[-1]).items():
        df0=close[loc:t1] # path prices
        df0=(df0/close[loc]-1)*events_.at[loc,'side'] # path returns
        out.loc[loc,'sl']=df0[df0<sl[loc]].index.min() # earliest stop loss
        out.loc[loc,'pt']=df0[df0>pt[loc]].index.min() # earliest profit taking
        
    return out


def parallel_apply(func, items, num_threads=4, **kwargs):
    
    def worker(molecule):
        return func(molecule=molecule, **kwargs)

    chunks = np.array_split(items, num_threads)
    
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        results = list(executor.map(worker, chunks))

    return pd.concat(results).sort_index()


def getEvents(close, tEvents, ptSl, trgt, minTrgt, numThreads=4, t1=False, side=None):
    # Step 1: Filter targets
    trgt = trgt.loc[tEvents]
    trgt = trgt[trgt > minTrgt]

    # Step 2: Set vertical barrier (t1)
    if t1 is False:
        t1 = pd.Series(pd.NaT, index=tEvents)

    # Step 3: Build events DataFrame
    if side is None:
        side_, ptSl_ = pd.Series(1., index=trgt.index), [ptSl[0], ptSl[0]]
    else:
        side_, ptSl_ = side.loc[trgt.index], ptSl[:2]

    events = pd.concat({'t1': t1, 'trgt': trgt, 'side': side_}, axis=1).dropna(subset=['trgt'])

    # Step 4: Apply barriers in parallel
    df0 = parallel_apply(
        func=applyPtSlOnT1,
        items=events.index,
        num_threads=numThreads,
        close=close,
        events=events,
        ptSl=ptSl_
    )

    # Step 5: Choose the first touched barrier
    events['t1'] = df0.dropna(how='all').min(axis=1)
    
    if side is None:
        events = events.drop('side', axis=1)
        
    return events


In [200]:
def get_vertical_barrier(t_events, close, delta=pd.Timedelta(minutes=60)):
    """
    For each event in t_events, find the timestamp in close.index 
    that is at least delta later. Returns a pd.Series of t1.
    """
    barrier_times = t_events + delta
    t1_idx = close.index.searchsorted(barrier_times)
    valid_idx = t1_idx[t1_idx < len(close)]
    t1 = pd.Series(close.index[valid_idx], index=t_events[:len(valid_idx)])
    return t1

In [201]:
t1 = get_vertical_barrier(t_events, df['close'], delta=pd.Timedelta(minutes=120))

In [202]:
t1

timestamp
2024-01-02 05:11:00   2024-01-02 07:11:00
2024-01-02 05:12:00   2024-01-02 07:12:00
2024-01-02 05:13:00   2024-01-02 07:13:00
2024-01-02 05:14:00   2024-01-02 07:14:00
2024-01-02 05:15:00   2024-01-02 07:15:00
                              ...        
2024-12-30 21:55:00   2024-12-30 23:55:00
2024-12-30 21:56:00   2024-12-30 23:56:00
2024-12-30 21:57:00   2024-12-30 23:57:00
2024-12-30 21:58:00   2024-12-30 23:58:00
2024-12-30 21:59:00   2024-12-30 23:59:00
Name: timestamp, Length: 102652, dtype: datetime64[ns]

In [203]:
trgt.describe()

count    102686.000000
mean          0.000160
std           0.000106
min           0.000017
25%           0.000087
50%           0.000135
75%           0.000199
max           0.001035
dtype: float64

In [204]:
minTrgt = trgt.quantile(.3)
minTrgt

np.float64(9.53639447054507e-05)

In [205]:
%%time

events = getEvents(
    close=df['close'],
    tEvents=t_events, 
    ptSl=[1,1], 
    trgt=trgt, 
    minTrgt=minTrgt, 
    numThreads=20, 
    t1=t1, 
    side=None
)

CPU times: user 19 s, sys: 1.26 s, total: 20.2 s
Wall time: 19.2 s


In [206]:
events.head()

,t1,trgt
timestamp,,
2024-01-02 05:11:00,2024-01-02 05:22:00,0.000105
2024-01-02 05:12:00,2024-01-02 05:15:00,0.000111
2024-01-02 05:13:00,2024-01-02 05:15:00,0.000111
2024-01-02 05:14:00,2024-01-02 05:22:00,0.000110
2024-01-02 05:15:00,2024-01-02 05:22:00,0.000115


In [207]:
events = events.dropna().copy()

# Keep only intraday events

In [208]:
events = events[events.index.date==events.t1.dt.date]

## Create classification labels

In [209]:
def getBins(events, close, minRet=0.0, t1=None):
    '''
    Compute event's outcome (including side information, if provided).
    events is a DataFrame where:
    -events.index is event's starttime
    -events['t1'] is event's endtime
    -events['trgt'] is event's target
    -events['side'] (optional) implies the algo's position side
    -t1 is original vertical barrier series
    Case 1: ('side' not in events): bin in (-1,1) <-label by price action
    Case 2: ('side' in events): bin in (0,1) <-label by pnl (meta-labeling)
    '''
    # 1) prices aligned with events
    events_ = events.dropna(subset=['t1'])
    px = events_.index.union(events_['t1'].values).drop_duplicates()
    px = close.reindex(px, method='bfill')
    # 2) create out object
    out = pd.DataFrame(index=events_.index)
    out['ret'] = px.loc[events_['t1'].values].values / px.loc[
        events_.index] - 1
    if 'side' in events_: out['ret'] *= events_['side']  # meta-labeling
    out['bin'] = np.sign(out['ret'])

    if 'side' not in events_:
        # only applies when not meta-labeling.
        # to update bin to 0 when vertical barrier is touched, we need the
        # original vertical barrier series since the events['t1'] is the time
        # of first touch of any barrier and not the vertical barrier
        # specifically. The index of the intersection of the vertical barrier
        # values and the events['t1'] values indicate which bin labels needs
        # to be turned to 0.
        vtouch_first_idx = events[events['t1'].isin(t1.values)].index
        out.loc[vtouch_first_idx, 'bin'] = 0.

    if 'side' in events_: out.loc[out['ret'] <= 0, 'bin'] = 0  # meta-labeling
    out.loc[out['ret'].abs() < minRet, 'bin'] = 0. # 不做獲利過少的交易 (設為 0)
    # out = out.loc[out['ret'].abs() >= minRet] # 過濾掉獲利過少的交易 (刪掉)
    return out

In [210]:
labels = getBins(events, df['close'], minRet=0, t1=t1)

In [211]:
print(labels.ret.abs().describe())
print(labels.ret.abs().quantile(.3))

count    71773.000000
mean         0.000266
std          0.000194
min          0.000009
25%          0.000166
50%          0.000222
75%          0.000307
max          0.004995
Name: ret, dtype: float64
0.0001767227053592002


In [212]:
labels.bin.value_counts()

bin
-1.0    26520
 1.0    25741
 0.0    19512
Name: count, dtype: int64

In [213]:
labels["bin_class"] = labels["bin"] + 1

In [214]:
labeled_events = events.join(labels, how='inner')

In [215]:
labeled_events.head()

,t1,trgt,ret,bin,bin_class
timestamp,,,,,
2024-01-02 05:11:00,2024-01-02 05:22:00,0.000105,0.000236,1.0,2.0
2024-01-02 05:12:00,2024-01-02 05:15:00,0.000111,0.000118,1.0,2.0
2024-01-02 05:13:00,2024-01-02 05:15:00,0.000111,0.000122,1.0,2.0
2024-01-02 05:14:00,2024-01-02 05:22:00,0.000110,0.000181,1.0,2.0
2024-01-02 05:15:00,2024-01-02 05:22:00,0.000115,0.000163,1.0,2.0


## Save events to disk

In [216]:
labeled_events.to_pickle(DIRECTION_LABEL_FILE_PATH)

In [217]:
DIRECTION_LABEL_FILE_PATH

PosixPath('../data/direction_labels/USDJPY-1m-20240101-20241231-Z-SCORE-W100-1Z-TB.pkl')